# patient

The patinet table is a core part of the eICU-CRD and contains all information related to tracking patient unit stays. The table also contains patient demographics and hospital level information.

In [3]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega

# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline

In [4]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: eicu


In [5]:
NUM_PATIENTS = 200859

## uniquePid

The `uniquePid` column identifies a single patient across multiple stays. Let's look at a single `uniquepid`.

In [6]:
uniquepid = '002-33870'
query = query_schema + """
select *
from patient
where uniquepid = '{}'
""".format(uniquepid)

df = pd.read_sql_query(query, con)
# df.to_excel("../outputs/single_patient.xlsx")
# df.to_csv("../outputs/single_patient.csv")
df.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141178,128927,Female,52,Caucasian,60,83,,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870
1,141179,128927,Female,52,Caucasian,60,83,,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870


Here we see two unit stays for a single patient. Note also that both unit stays have the same `patienthealthsystemstayid` - this indicates that they occurred within the same hospitalization.

We can see the `unitstaytype` was 'admit' for one stay, and 'stepdown/other' for another. Other columns can give us more information.

In [7]:
df[['patientunitstayid', 'wardid', 'unittype', 'unitstaytype', 'hospitaladmitoffset', 'unitdischargeoffset']]

,patientunitstayid,wardid,unittype,unitstaytype,hospitaladmitoffset,unitdischargeoffset
0,141178,83,Med-Surg ICU,admit,-14,8
1,141179,83,Med-Surg ICU,stepdown/other,-22,2042


Note that it's not explicitly obvious which stay occurred first. Earlier stays will be closer to hospital admission, and therefore have a *higher* hospitaladmitoffset. Above, the stay with a `hospitaladmitoffset` of -14 was first (occurring 14 minutes after hospital admission), followed by the next stay with a `hospitaladmitoffset` of 22 (which occurred 22 minutes after hospital admission). Practically, we wouldn't consider the first admission a "real" ICU stay, and it's likely an idiosyncrasy of the administration system at this particular hospital. Notice how both rows have the same `wardid`.

## Age

As ages over 89 are required to be deidentified by HIPAA, the `age` column is actually a string field, with ages over 89 replaced with the string value '> 89'.

In [8]:
query = query_schema + """
select age, count(*) as n
from patient
group by age
order by n desc
"""

df = pd.read_sql_query(query, con)
df.head()

,age,n
0,> 89,7081
1,67,5078
2,68,4826
3,72,4804
4,71,4764


As is common in eICU-CRD, there are a subset of hospitals who routinely utilize this portion of the medical record (and thus have 90-100% data completion), while there are other hospitals who rarely use this interface and thus have poor data completion (0-10%).

## unitdischargestatus
Identifies whether the patient is alive or dead when discharged from ICU.  
There is a similar one for hospital discharge.

In [9]:
query = query_schema + """
select unitdischargestatus, count(*) as n
from patient
group by unitdischargestatus
order by n desc
"""

df = pd.read_sql_query(query, con)
df.head()

,unitdischargestatus,n
0,Alive,189918
1,Expired,10907
2,,34


In [10]:
unitdischargestatus = 'Expired'
query = query_schema + """
select *
from patient
where unitdischargestatus = '{}'
order by patienthealthsystemstayid
""".format(unitdischargestatus)

df = pd.read_sql_query(query, con)
df.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141297,129026,Male,63,Caucasian,73,85,"Sepsis, pulmonary",162.6,04:18:00,...,Floor,2,readmit,NaN,74.4,03:41:00,1869,Death,Expired,002-30269
2,141314,129039,Male,45,Caucasian,73,85,"Aneurysm, abdominal aortic; with rupture",170.2,00:41:00,...,Operating Room,2,transfer,NaN,102.6,18:39:00,632,Death,Expired,002-70742
3,141556,129238,Female,83,Caucasian,56,82,,165.1,22:54:00,...,ICU to SDU,2,stepdown/other,NaN,NaN,14:19:00,2459,Death,Expired,002-35104
4,142154,129683,Male,74,Caucasian,73,97,,167.6,17:29:00,...,ICU to SDU,2,stepdown/other,NaN,NaN,19:32:00,1081,Death,Expired,002-7333


## patienthealthsystemstayid
Entries with the same ID indicate that there is multiple ICU visits during the same hospitalization.

In [11]:
query = query_schema + """
select patienthealthsystemstayid, count(*) as n
from patient
group by patienthealthsystemstayid
order by n desc
"""

df = pd.read_sql_query(query, con)
df.head()

,patienthealthsystemstayid,n
0,590180,17
1,498628,13
2,2416711,12
3,631956,11
4,603432,11


In [12]:
patienthealthsystemstayid = '590180'
query = query_schema + """
select *
from patient
where patienthealthsystemstayid = '{}'
order by hospitaladmitoffset
""".format(patienthealthsystemstayid)

df = pd.read_sql_query(query, con)
df.head()
df[['patientunitstayid', 'wardid', 'unittype', 'unitstaytype', 'hospitaladmitoffset', 'unitdischargeoffset']]

,patientunitstayid,wardid,unittype,unitstaytype,hospitaladmitoffset,unitdischargeoffset
0,766322,374,Med-Surg ICU,stepdown/other,-81470,8815
1,766315,374,Med-Surg ICU,readmit,-71603,9867
2,766308,374,Med-Surg ICU,stepdown/other,-71002,601
3,766311,374,Med-Surg ICU,readmit,-70067,935
4,766312,374,Med-Surg ICU,stepdown/other,-58401,2269
5,766317,374,Med-Surg ICU,readmit,-55207,3194
6,766321,374,Med-Surg ICU,stepdown/other,-46750,8457
7,766310,374,Med-Surg ICU,readmit,-42670,4080
8,766320,374,Med-Surg ICU,stepdown/other,-42613,57
9,766319,374,Med-Surg ICU,readmit,-42612,1


## unitType
Type of ICU the patients stayed in.

In [13]:
query = query_schema + """
select unitType, count(*) as n
from patient
group by unitType
order by n desc
"""

df = pd.read_sql_query(query, con)
assert df['n'].sum() == NUM_PATIENTS, f"{df['n'].sum()} =/= {NUM_PATIENTS}"
df

,unittype,n
0,Med-Surg ICU,113222
1,MICU,17465
2,CCU-CTICU,15290
3,Neuro ICU,14451
4,Cardiac ICU,12467
5,SICU,12181
6,CSICU,9625
7,CTICU,6158
